In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import DeepfakeDataset
from model import EfficientNetModel
from config import Config
import pandas as pd

def test_model():
    transform = transforms.Compose([
        transforms.Resize((Config.img_size, Config.img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    test_dataset = DeepfakeDataset(Config.test_path, transform=transform, is_test=True)
    test_loader = DataLoader(test_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=Config.num_workers, pin_memory=Config.pin_memory)
    
    model = EfficientNetModel().to(Config.device)
    model.load_state_dict(torch.load('best_model_fold0.pth', map_location=Config.device))
    model.eval()
    
    results = []
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(Config.device)
            outputs = model(images).squeeze(1)
            predictions = torch.sigmoid(outputs).cpu().numpy()
            
            for img_id, pred in zip(image_ids, predictions):
                results.append({'id': img_id, 'label': int(pred > 0.5)})
    
    df = pd.DataFrame(results)
    df.to_csv('submission.csv', index=False)
    print("Predictions saved to submission.csv")

if __name__ == "__main__":
    print("Starting testing...")
    test_model()
